In [65]:
import pandas as pd
import numpy as np
import json
from scipy.spatial.distance import cosine
from sklearn.feature_extraction.text import CountVectorizer
tokenizer = CountVectorizer().build_tokenizer()

In [69]:
w2v_path = "c:/workspace/blog_stuff/classification_w2v/glove.840B.300d.txt"
# w2v_path = "c:/workspace/blog_stuff/classification_w2v/glove.6B.50d.txt"
train_path = "training_set.tsv"
test_path = "validation_set.tsv"

In [70]:
df = pd.read_csv(train_path, delimiter="\t")

In [71]:
w2v = {}
with open(w2v_path, "rb") as infile:
    for line in infile:
        parts = line.split()
        w = parts[0]
        v = np.array([float(x) for x in parts[1:]])
        w2v[w] = v
dim = len(w2v.itervalues().next())

In [85]:
stopwords = "a, a's, able, about, above, according, accordingly, across, actually, after, afterwards, again, against, ain't, all, allow, allows, almost, alone, along, already, also, although, always, am, among, amongst, an, and, another, any, anybody, anyhow, anyone, anything, anyway, anyways, anywhere, apart, appear, appreciate, appropriate, are, aren't, around, as, aside, ask, asking, associated, at, available, away, awfully, be, became, because, become, becomes, becoming, been, before, beforehand, behind, being, believe, below, beside, besides, best, better, between, beyond, both, brief, but, by, c'mon, c's, came, can, can't, cannot, cant, cause, causes, certain, certainly, changes, clearly, co, com, come, comes, concerning, consequently, consider, considering, contain, containing, contains, corresponding, could, couldn't, course, currently, definitely, described, despite, did, didn't, different, do, does, doesn't, doing, don't, done, down, downwards, during, each, edu, eg, eight, either, else, elsewhere, enough, entirely, especially, et, etc, even, ever, every, everybody, everyone, everything, everywhere, ex, exactly, example, except, far, few, fifth, first, five, followed, following, follows, for, former, formerly, forth, four, from, further, furthermore, get, gets, getting, given, gives, go, goes, going, gone, got, gotten, greetings, had, hadn't, happens, hardly, has, hasn't, have, haven't, having, he, he's, hello, help, hence, her, here, here's, hereafter, hereby, herein, hereupon, hers, herself, hi, him, himself, his, hither, hopefully, how, howbeit, however, i'd, i'll, i'm, i've, ie, if, ignored, immediate, in, inasmuch, inc, indeed, indicate, indicated, indicates, inner, insofar, instead, into, inward, is, isn't, it, it'd, it'll, it's, its, itself, just, keep, keeps, kept, know, knows, known, last, lately, later, latter, latterly, least, less, lest, let, let's, like, liked, likely, little, look, looking, looks, ltd, mainly, many, may, maybe, me, mean, meanwhile, merely, might, more, moreover, most, mostly, much, must, my, myself, name, namely, nd, near, nearly, necessary, need, needs, neither, never, nevertheless, new, next, nine, no, nobody, non, none, noone, nor, normally, not, nothing, novel, now, nowhere, obviously, of, off, often, oh, ok, okay, old, on, once, one, ones, only, onto, or, other, others, otherwise, ought, our, ours, ourselves, out, outside, over, overall, own, particular, particularly, per, perhaps, placed, please, plus, possible, presumably, probably, provides, que, quite, qv, rather, rd, re, really, reasonably, regarding, regardless, regards, relatively, respectively, right, said, same, saw, say, saying, says, second, secondly, see, seeing, seem, seemed, seeming, seems, seen, self, selves, sensible, sent, serious, seriously, seven, several, shall, she, should, shouldn't, since, six, so, some, somebody, somehow, someone, something, sometime, sometimes, somewhat, somewhere, soon, sorry, specified, specify, specifying, still, sub, such, sup, sure, t's, take, taken, tell, tends, th, than, thank, thanks, thanx, that, that's, thats, the, their, theirs, them, themselves, then, thence, there, there's, thereafter, thereby, therefore, therein, theres, thereupon, these, they, they'd, they'll, they're, they've, think, third, this, thorough, thoroughly, those, though, three, through, throughout, thru, thus, to, together, too, took, toward, towards, tried, tries, truly, try, trying, twice, two, un, under, unfortunately, unless, unlikely, until, unto, up, upon, us, use, used, useful, uses, using, usually, value, various, very, via, viz, vs, want, wants, was, wasn't, way, we, we'd, we'll, we're, we've, welcome, well, went, were, weren't, what, what's, whatever, when, whence, whenever, where, where's, whereafter, whereas, whereby, wherein, whereupon, wherever, whether, which, while, whither, who, who's, whoever, whole, whom, whose, why, will, willing, wish, with, within, without, won't, wonder, would, would, wouldn't, yes, yet, you, you'd, you'll, you're, you've, your, yours, yourself, yourselves, zero"
stopwords = set(stopwords.split(", "))

In [99]:
df = pd.read_csv(test_path, delimiter="\t")

In [100]:
def vectorizer(text):
    tokens = filter(lambda x: x not in stopwords, tokenizer(text))
    vecs = np.array([w2v[w.lower()] for w in tokens if w.lower() in w2v])
    if len(vecs) == 0:
        return np.zeros(dim)
    return np.mean(vecs, axis=0)

In [101]:
Q = df.question.map(vectorizer)
A = df.answerA.map(vectorizer)
B = df.answerB.map(vectorizer)
C = df.answerC.map(vectorizer)
D = df.answerD.map(vectorizer)

In [102]:
pred = []
for q, a, b, c, d in zip(Q, A, B, C, D):
    answer = min([('a', a), ('b', b), ('c', c), ('d', d)], key=lambda (_, v): cosine(q, v))[0].upper()
    pred.append(answer)
pred = np.array(pred)

In [90]:
(pred == df.correctAnswer).mean()

0.30840000000000001

In [39]:
vecs.mean(axis=0)

array([ 0.4582206 ,  0.239866  ,  0.3553994 ,  0.3579928 ,  0.3073258 ,
       -0.2184158 , -0.470578  , -0.55869   ,  0.595454  , -0.254467  ,
       -0.0076742 ,  0.3721934 , -0.1770902 , -0.0689644 , -0.2461286 ,
        0.0050664 , -0.2583322 ,  0.1195142 ,  0.388638  , -0.17244   ,
        0.766538  ,  0.3358442 ,  0.155528  ,  0.0096628 , -0.1381356 ,
       -1.479698  , -0.409821  , -0.436258  ,  0.0247856 ,  0.0591946 ,
        3.33572   , -0.159728  , -0.06381   , -1.05191   ,  0.0303708 ,
        0.329164  , -0.513948  ,  0.2547698 ,  0.0417104 ,  0.06354122,
        0.4818934 ,  0.0753122 ,  0.381492  ,  0.2256132 ,  0.0378272 ,
       -0.0973778 , -0.1258668 ,  0.839568  ,  0.16969   ,  0.331012  ])

In [104]:
df["correctAnswer"] = pred

In [106]:
df.to_csv("answers.csv", columns=["id", "correctAnswer"],index=False)

In [91]:
df_test = pd.read_csv(test_path, delimiter="\t")

In [94]:
df_test.question[0]

'A meter is industry by developing a process of heating and cooling are examples of what body system?'

In [98]:
df_test.question[2]

'Which of the seeds grow better in the light or the that provides a framework of of its organs?'

In [97]:
df.question[0]


'When athletes begin to exercise, their heart rates and respiration rates increase.  At what level of organization does the human body coordinate these functions?'